# Brace Analysis
The photo shows the enf\d details of a typical brace in a 4-storey steel structure.  This was photographed in Ottawa, in September, 2015.

This notebook shows all of the computations necessary to compute $T_r$ for a similar brace.  **Caution**, all of the dimensions and properties were invented by the author of this notebook; no attempt has been made to have an accurate model of the real structure.

![Brace End Details](brace.jpg)

![Overall Sketch](brace-sketch.svg)

In [1]:
from Designer import Designer, Data
import math
sind = lambda t: math.sin(math.radians(t))
cosd = lambda t: math.cos(math.radians(t))

In [2]:
import sst
SST = sst.SST()

In [3]:
phiw = 0.67
phiu = 0.75
phib = 0.80
phibr = 0.80
phi = 0.90

In [4]:
des = Designer(title='Typical Cross Brace',var='Tr',units='kN',trace=True)

# useful abbreviations:
REQ = des.require       # a requirement
CHK = des.check         # a check
REC = des.record        # record calculation details

In [5]:
HSS = Data(Grade='CSA G40.21 350W',Fy=350.,Fu=450.,size='HS127x127x13')
PL = Data(Grade='CSA G40.21 350W',Fy=350.,Fu=450.)
Bolt = Data(Grade='ASTM A325M',Size='M20',d=20.,Fu=830.,threads_intercepted=True)
Bolt.hd = Bolt.d+2+2
Weld = Data(Class='E49XX',Xu=490.,matching=True,)
Cover = Data(t=10,w=60,Grade='CSA G40.21 350W',Fy=350.,Fu=450.,D=6.)

## Gusset Plate Details
![Gusset Plate Details](gusset-details.svg)

In [6]:
Gusset = Data(w1=280.,    # width normal to axis
              w2=110.,    # distance from face of col to start of w1
              theta=45.,  # angle to vertical
              t = 20.,    # thickness
              D = 6.,     # weld leg thickness
              nlines = 2, # number of transverse lines of bolts (a line is perpendicular to axis/force)
              nacross = 3,  # number of rows of bolts across, or bolts per line
              e = 40.,    # end distance - first line to edge of gusset
              g = 75.,    # gauge 
              s = 75.,    # spacing
             ) 

### Gusset to HSS Weld

In [7]:
w1,w2,D,theta = Gusset['w1,w2,D,theta']
Xu = Weld['Xu']

L1 = w2+w1*cosd(theta)
L = (L1/sind(theta))*cosd(theta) + w1*sind(theta)

Mw = 1.0
Aw = 2*L*.707*D
Vr = 0.67*phiw*Aw*Xu*(1+sind(theta)**1.5)*Mw * 1E-3

REC('Gusset to HSS Weld','L,Xu,w1,w2,D,theta,Aw,Mw,phiw',Tr=Vr)

    Gusset to HSS Weld: Tr = 1506 kN
       (Aw=4293, D=6.0, L=506.0, Mw=1.0, Xu=490.0, phiw=0.67, theta=45.0, w1=280.0, w2=110.0)


### Gusset Block Shear
#### Case 1)

In [8]:
nlines,nacross,g,s,e,t = Gusset['nlines,nacross,g,s,e,t']
hd = Bolt['hd']
Fy,Fu = PL['Fy,Fu']

An = t*((nacross-1)*g - (nacross-1)*hd)
Agv = 2*t*((nlines-1)*s + e)
Ut = 1.0
Vr = phiu*(Ut*An*Fu + 0.6*Agv*(Fy+Fu)/2.) * 1E-3
REC('Gusset Block Shear 1)','An,Agv,Ut',Tr=Vr)

    Gusset Block Shear 1): Tr = 1516 kN
       (Agv=4600, An=2040, Ut=1.0)


#### Case 2)

In [9]:
An = t*((nacross-1)*g + ((w1/2.)-(nacross-1)*g/2.))
Agv = t*((nlines-1)*s + e)
Ut = 0.8
Vr = phiu*(Ut*An*Fu + 0.6*Agv*(Fy+Fu)/2.) * 1E-3
REC('Gusset Block Shear 2)','An,Agv,Ut',Tr=Vr)

    Gusset Block Shear 2): Tr = 1575 kN
       (Agv=2300, An=4300, Ut=0.8)


#### Tearout

In [10]:
Agv = t*((nlines-1)*s + e) * nacross * 2
Vr = phiu * 0.6*Agv*(Fy+Fu)/2. * 1E-3
REC('Gusset tearout','Agv',Tr=Vr)

    Gusset tearout: Tr = 2484 kN
       (Agv=13800)


## Lap Plate
### Lap Plate Details
Consider the thickness of both plates together when computing the resistance.
As the plate is symmetric, we only have to investigate one end.
![Lap Plate Details](lap-plate-details.svg)

In [11]:
LapPlate = Data(W = 230.,
                L = 315.,
                t = 10.+10.,     # include 2 plates
                e = 40.,         # could be different than gusset
                g = Gusset.g,    # must be the same as gusset plate
                s = Gusset.s,
                nlines = Gusset.nlines,
                nacross = Gusset.nacross,
               )

### Lap Plate: Gross Section Yielding

In [12]:
wg,t = LapPlate['W,t']
Fy = PL.Fy
Ag = wg*t
Tr = phi*Ag*Fy * 1E-3
REC('Lap Plate, Gross Yield','Ag,Fy,phi',Tr=Tr)

    Lap Plate, Gross Yield: Tr = 1449 kN
       (Ag=4600, Fy=350.0, phi=0.9)


### Lap Plate: Net Section Fracture

In [13]:
wg,t,nacross = LapPlate['W,t,nacross']
hd = Bolt.hd
Fu = PL.Fu

wn = wg - nacross*hd
Ane = An = wn*t
Tr = phiu*Ane*Fu * 1E-3
REC('Lap Plate, Net Fracture','Fu,phiu,Ane',Tr=Tr)

    Lap Plate, Net Fracture: Tr = 1066 kN
       (Ane=3160, Fu=450.0, phiu=0.75)


### Lap Plate: Block Shear
#### Case 1)

In [14]:
g,t,nacross,nlines,e,s = LapPlate['g,t,nacross,nlines,e,s']
hd = Bolt.hd
Fy,Fu = PL['Fy,Fu']

An = ((nacross-1)*g - (nacross-1)*hd)*t
Agv = (e + (nlines-1)*s)*t*2
Ut = 1.0
Vr = phiu*(Ut*An*Fu + 0.6*Agv*(Fy+Fu)/2.) * 1E-3
REC('Lap Plate Block Shear 1)','An,Agv,Ut',Tr=Vr)

    Lap Plate Block Shear 1): Tr = 1516 kN
       (Agv=4600, An=2040, Ut=1.0)


#### Case 2)

In [15]:
g,t,nacross,nlines,e,s,wg = LapPlate['g,t,nacross,nlines,e,s,W']
hd = Bolt.hd
Fy,Fu = PL['Fy,Fu']

edge = (wg - (nacross-1)*g)/2.0
An = (wg - edge - (nacross-0.5)*hd)*t
Agv = (e + (nlines-1)*s)*t
Ut = 0.8
Vr = phiu*(Ut*An*Fu + 0.6*Agv*(Fy+Fu)/2.) * 1E-3
REC('Lap Plate Block Shear 2)','An,Agv,Ut',Tr=Vr)

    Lap Plate Block Shear 2): Tr = 1116 kN
       (Agv=2300, An=2600, Ut=0.8)


#### Tearout

In [16]:
t,nlines,nacross,e,s = LapPlate['t,nlines,nacross,e,s']
Fy,Fu = PL['Fy,Fu']

An = 0
Agv = (e + (nlines-1)*s)*t*2*nacross
Ut = 1
Vr = phiu*(Ut*An*Fu + 0.6*Agv*(Fy+Fu)/2.) * 1E-3
REC('Lap Plate tearout','Agv',Tr=Vr)

    Lap Plate tearout: Tr = 2484 kN
       (Agv=13800)


## Tongue Plate
### Tongue Plate Details
![Tongue Plate Details](tongue-plate-details.svg)

In [17]:
Tongue = Data(W = Gusset.w1,
              t = Gusset.t,
              D = 8.,
              nlines = Gusset.nlines,
              nacross = Gusset.nacross,
              e = 40.,
              s = Gusset.s,
              g = Gusset.g,
              c = 45.,
              L = 260.              
             )
Tongue.Lw = Tongue.L - (Tongue.c + (Tongue.nlines-1)*Tongue.s + Tongue.e)
Tongue.Dh = SST.section(HSS.size,'D')

In [18]:
vars(Tongue)

{'D': 8.0,
 'Dh': 127.0,
 'L': 260.0,
 'Lw': 100.0,
 'W': 280.0,
 'c': 45.0,
 'e': 40.0,
 'g': 75.0,
 'nacross': 3,
 'nlines': 2,
 's': 75.0,
 't': 20.0}

### Tongue Plate - Bolted End
#### Gross Section Yield

In [19]:
wg,t = Tongue['W,t']
Fy = PL['Fy']
Ag = wg*t
Tr = phi*Ag*Fy * 1E-3
REC('Tongue Plate, Gross Yield','Ag',Tr=Tr)

    Tongue Plate, Gross Yield: Tr = 1764 kN
       (Ag=5600)


#### Net Section Fracture

In [20]:
wg,t,n = Tongue['W,t,nacross']
Fu = PL['Fu']
hd = Bolt['hd']

wn = wg - n*hd
Ane = An = wn*t
Tr = phiu*An*Fu * 1E-3
REC('Tongue Plate, Net Area Fracture (Bolted)','An',Tr=Tr)

    Tongue Plate, Net Area Fracture (Bolted): Tr = 1404 kN
       (An=4160)


#### Block Shear
There is no need to check block shear.  Of necessity, the thickness of the tongue plate is the same as
the gusset plate, and as the widths and bolting details are also the same, the capacities with respect to block shear
will also be the same.

### Tongue Plate - Welded End
#### Shear Lag - Effective Net Area

In [21]:
w,w2,L,t = Tongue['W,Dh,Lw,t']
Fu = PL.Fu
if L >= 2*w2:
    An2 = 1.00*w2*t
elif L >= w2:
    An2 = 0.5*w2*t + 0.25*L*t
else:
    An2 = 0.75*L*t
    
n,g = Tongue['nacross,g']
w3 = (w-w2)/2.
xbar = w3/2.
if L >= w3:
    An3 = (1.-xbar/L)*w3*t
else:
    An3 = 0.50*L*t
Ane = An2 + An3 + An3
Tr = phiu*Ane*Fu * 1E-3
REC('Tongue Plate, Net Area Fracture (Welded)','w2,w3,An2,An3,Ane',Tr=Tr)

    Tongue Plate, Net Area Fracture (Welded): Tr = 1144 kN
       (An2=1500, An3=944.8, Ane=3390, w2=127.0, w3=76.5)


## Fasteners
### Bolts
#### Shear

In [22]:
d,Fu,threads_intercepted = Bolt['d,Fu,threads_intercepted']
t = min(Gusset.t,Tongue.t,2*LapPlate.t)
n = min(Gusset.nlines*Gusset.nacross,Tongue.nlines*Tongue.nacross)
m = 2
Ab = 3.14159*d*d/4.
Vr = 0.6*phib*n*m*Ab*Fu * 1E-3
if threads_intercepted:
    Vr *= 0.7
REC('Bolts in Shear','n,m,d,Ab,Fu,t',Tr=Vr)

    Bolts in Shear: Tr = 1051 kN
       (Ab=314.2, Fu=830.0, d=20.0, m=2, n=6, t=20.0)


#### Bearing

In [23]:
Br = 3*phibr*n*t*d*Fu * 1E-3
REC('Bolts in Bearing','n,t,d,Fu',Tr=Br)

    Bolts in Bearing: Tr = 4781 kN
       (Fu=830.0, d=20.0, n=6, t=20.0)


### Welds - HSS to Tongue Plate

In [24]:
L,D = Tongue['Lw,D']
Xu,matching = Weld['Xu,matching']
Aw = 4.*L*D*0.707
Vr = 0.67*phiw*Aw*Xu * 1E-3
if matching:
    REC('Weld metal (HSS to Plate)','D,L,Aw,Xu',Tr=Vr)
else:
    Fu = min(HSS.Fu,PL.Fu)
    Am = 4.*L*D
    Vr = min(Vr,0.67*phiw*Am*Fu)
    REC('Weld/Base metal (HSS to Plate)','D,L,Aw,Am,Xu,Fu',Tr=Vr)

    Weld metal (HSS to Plate): Tr = 497.6 kN
       (Aw=2262, D=8.0, L=100.0, Xu=490.0)


## HSS + Cover Plate
![HSS Details](hss-details.svg)
Each half of the HSS is a plate of width w (the meridian distance around) welded on 2 edges (perpendicular
to the figure).  The cover plate is an additional plate welded on two edges.
### Net Section Fracture

In [25]:
D,t,Ag = SST.section(HSS.size,'D,T,A')
hFy,hFu = HSS['Fy,Fu']
tt,L = Tongue['t,Lw']
w = (4.*(D-t) - 2*tt)/2.   # meridian distance around 1/2 of cut HSS
if L >= 2*w:
    An2 = 1.00*w*t
elif L >= w:
    An2 = 0.5*w*t + 0.25*L*t
else:
    An2 = 0.75*L*t
hAne = An2
hTr = 2*phiu*hAne*hFu * 1E-3

cw,ct,cFy,cFu = Cover['w,t,Fy,Fu']  # cover plate
if L >= 2*cw:
    cAn2 = 1.00*cw*ct
elif L >= cw:
    cAn2 = 0.5*cw*ct + 0.25*L*ct
else:
    cAn2 = 0.75*L*ct
cAne = cAn2
cTr = 2*phiu*cAne*cFu * 1E-3
Tr = hTr + cTr
REC('HSS, net section fracture','w,t,hAne,hTr,cw,cAne,cTr',Tr=Tr)

    HSS, net section fracture: Tr = 1014 kN
       (cAne=550.0, cTr=371.2, cw=60, hAne=952.5, hTr=642.9, t=12.7, w=208.6)


### Gross Section Yield

In [26]:
Tr = phi*Ag*Fy  * 1E-3
REC('HSS Gross Section Yield','Ag',Tr=Tr)

    HSS Gross Section Yield: Tr = 1698 kN
       (Ag=5390)


## Cover Plate to HSS Weld
Ensure that the length of the weld can develop the full strength of the cover plate.

In [27]:
cTr = phiu*cAne*cFu * 1E-3
L = Tongue.Lw
D = Cover.D
Xu = Weld.Xu
Aw = 2*L*D*0.707
wTr = 0.67*phiw*Aw*Xu * 1E-3
CHK(wTr>=cTr,'Coverplate weld strength'.format(wTr,cTr),'wTr,cTr,L,D,Aw')

    Coverplate weld strength:  OK - (wTr=186.6, cTr=185.6, L=100.0, D=6.0, Aw=848.4)


## To Do
* Check that gusset plate doesn't cause undo flexural problems in the flange of the HSS column member.

## Summary

In [28]:
des.summary()


Summary of Tr for Designer: Typical Cross Brace

Checks:
-------
    Coverplate weld strength:   OK - (wTr=186.6, cTr=185.6, L=100.0, D=6.0, Aw=848.4)

Values of Tr:
-------------
    Gusset to HSS Weld:                       Tr = 1510 kN
       (Aw=4293, D=6.0, L=506.0, Mw=1.0, Xu=490.0, phiw=0.67, theta=45.0, w1=280.0, w2=110.0)
    Gusset Block Shear 1):                    Tr = 1520 kN
       (Agv=4600, An=2040, Ut=1.0)
    Gusset Block Shear 2):                    Tr = 1580 kN
       (Agv=2300, An=4300, Ut=0.8)
    Gusset tearout:                           Tr = 2480 kN
       (Agv=13800)
    Lap Plate, Gross Yield:                   Tr = 1450 kN
       (Ag=4600, Fy=350.0, phi=0.9)
    Lap Plate, Net Fracture:                  Tr = 1070 kN
       (Ane=3160, Fu=450.0, phiu=0.75)
    Lap Plate Block Shear 1):                 Tr = 1520 kN
       (Agv=4600, An=2040, Ut=1.0)
    Lap Plate Block Shear 2):                 Tr = 1120 kN
       (Agv=2300, An=2600, Ut=0.8)
    Lap Plate tearo